In [1]:
import json
import urllib
import math
import pandas as pd 
import numpy as np

In [2]:
x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626  # π
a = 6378245.0  # 长半轴
ee = 0.00669342162296594323  # 偏心率平方

In [3]:
def gcj02_to_wgs84(lng, lat):
    """
    GCJ02(火星坐标系)转GPS84
    :param lng:火星坐标系的经度
    :param lat:火星坐标系纬度
    :return:GPS84坐标
    """
    if out_of_china(lng, lat):
        return [lng, lat]
    dlat = _transformlat(lng - 105.0, lat - 35.0)
    dlng = _transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [lng * 2 - mglng, lat * 2 - mglat]

In [4]:
def _transformlat(lng, lat):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + \
          0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * pi) + 40.0 *
            math.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * pi) + 320 *
            math.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret


def _transformlng(lng, lat):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
          0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * pi) + 40.0 *
            math.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * pi) + 300.0 *
            math.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret

In [5]:
def out_of_china(lng, lat):
    """
    判断是否在国内，不在国内不做偏移
    :param lng:经度
    :param lat:纬度
    :return:bool布尔值
    """
    return not (lng > 73.66 and lng < 135.05 and lat > 3.86 and lat < 53.55)

In [6]:
gcj02_to_wgs84(118.784129,32.041806)

[118.77892757265252, 32.0438602407582]

In [7]:
df_sts = pd.read_csv('station.csv')

In [8]:
df_sts.head(1)

,Unnamed: 0,station_coords,station_names
0,0,"'118.777761,31.981484'",阅城国际南门


In [9]:
df_sts['station_coords'][0].replace('\'','')

'118.777761,31.981484'

In [10]:
float(df_sts['station_coords'][0].replace('\'','').split(',')[0])

118.777761

In [11]:
def st_wgs84(x):
    lng=float(x.replace('\'','').split(',')[0])
    lat=float(x.replace('\'','').split(',')[1])
    return gcj02_to_wgs84(lng,lat)

In [12]:
df_sts['station_coords_wgs84']= df_sts['station_coords'].apply(st_wgs84)

In [13]:
df_sts.head(1)

,Unnamed: 0,station_coords,station_names,station_coords_wgs84
0,0,"'118.777761,31.981484'",阅城国际南门,"[118.77255737606791, 31.98351748799738]"


In [14]:
import shapefile

In [15]:
df_sts['station_coords_wgs84'][0]

[118.77255737606791, 31.98351748799738]

In [16]:
len(df_sts)

3007

In [17]:
w=shapefile.Writer(r'D:\PycharmProjects\pythonProject9\TransportData\nanjing_station.shp')
w.field('name','C')
for i in range(len(df_sts)):
    w.point(df_sts['station_coords_wgs84'][i][0],df_sts['station_coords_wgs84'][i][1])
    w.record(df_sts['station_names'][i])
w.close()

In [19]:
df_ls = pd.read_json('lines.json')

In [20]:
df_ls.head(1)

,line_name,lines
0,X1路(阅城国际南门--汇康路),"[118.777761,31.981484, 118.777913,31.981501, 1..."


In [21]:
df_ls['lines'][0]

['118.777761,31.981484',
 '118.777913,31.981501',
 '118.77837,31.98156',
 '118.778766,31.981619',
 '118.778965,31.981651',
 '118.779078,31.981673',
 '118.779442,31.981737',
 '118.779952,31.981855',
 '118.780129,31.981898',
 '118.780392,31.981957',
 '118.781213,31.982215',
 '118.782125,31.98259',
 '118.782511,31.982751',
 '118.782784,31.982874',
 '118.783739,31.983276',
 '118.784179,31.983464',
 '118.784844,31.983743',
 '118.784985,31.983802',
 '118.785241,31.98391',
 '118.785826,31.984167',
 '118.786421,31.984425',
 '118.786502,31.984457',
 '118.786845,31.984596',
 '118.787064,31.984688',
 '118.78743,31.984848',
 '118.787703,31.984961',
 '118.78824,31.985186',
 '118.788342,31.985235',
 '118.788707,31.985385',
 '118.789533,31.985739',
 '118.789656,31.985798',
 '118.790375,31.986125',
 '118.790783,31.986308',
 '118.791469,31.986597',
 '118.792113,31.986881',
 '118.792564,31.987085',
 '118.792687,31.98714',
 '118.793261,31.987396',
 '118.793486,31.987498',
 '118.794178,31.987799',
 '118.7

In [25]:
def lines_wgs84(x):
    lst =[]
    for i in x:
        lng=float(i.replace('\'','').split(',')[0])
        lat=float(i.replace('\'','').split(',')[1])
        lst.append(gcj02_to_wgs84(lng,lat))
    return lst

In [26]:
df_ls['lines_wgs84']= df_ls['lines'].apply(lines_wgs84)

In [27]:
df_ls.head(1)

,line_name,lines,lines_wgs84
0,X1路(阅城国际南门--汇康路),"[118.777761,31.981484, 118.777913,31.981501, 1...","[[118.77255737606791, 31.98351748799738], [118..."


In [29]:
w= shapefile.Writer(r'D:\PycharmProjects\pythonProject9\TransportData\nanjing_lines.shp')
w.field('name','C')
for i in range(len(df_ls)):
    w.line([df_ls['lines_wgs84'][i]])
    w.record(df_ls['line_name'][i])
w.close()